(C:filtre-numerique)=
# Filtres numériques

Les filtres numériques sont très courants et utilisés dans de nombreux domaines :
télécommunications, traitement de la parole, Hi-Fi, radar et sonar, asservissement, vidéo, photos, modem, etc.

S'ils sont à ce point répandus, c'est en raison de leurs nombreux avantages par rapport aux filtres analogiques :
* deux filtres numériques de même configuration sont parfaitement identiques,
* un filtre numérique est facilement reprogrammable,
* l'association de filtres numériques ne pose pas de problème d'impédance,
* il n'y a pas de variation des caractéristiques en fonction de l'âge ou de la température des composants,
* on dispose d'une grande précision de calcul.

En revanche, les filtres numériques sont bien entendu limités par leur fréquence d'échantillonnage
et naturellement inadaptés au traitement du signal analogique.

La réponse impulsionnelle d'un filtre numérique est échantillonnée à une fréquence d'échantillonnage $f_e$.
Dans ce cours, on traitera uniquement des filtres causaux :

$$
h[n] = 0\;\text{si}\; n<0.
$$

Par ailleurs, lorsque la réponse impulsionnelle est à durée limitée, c'est à dire $h[n]=0$ si $n > N$,
alors le filtre est à réponse impulsionnelle finie (RIF).
Dans le cas contraire, on parle de filtre à réponse impulsionnelle infinie (RII).
Ces deux types de filtres sont présentés ci-après, ainsi que deux méthodes de conception :
* la méthode des fenêtres pour les filtres RIF,
* la transformation bilinéaire pour les filtres RII.

## Filtres RIF


### Réponse impulsionnelle

Un filtre RIF a pour réponse impulsionnelle :

$$
h[n] =
\begin{cases}
b_n &\text{si}\, n \in \{0,\dots,N\}, \\
0   &\text{sinon}.
\end{cases}
$$

Sa réponse impulsionnelle est finie car elle possède $N+1$ coefficients non nuls :
on dit que le filtre est de longueur $N+1$, ou d'ordre $N$.

### Fonction de transfert

La transformée en Z de $h$ donne la fonction de transfert :

$$
H(z) = \sum_{n=-\infty}^{+\infty} h[n] z^{-n} = \sum_{n=0}^{N} b_n z^{-n}.
$$

Cette fonction de transfert est particulière car elle ne possède pas de pôle,
autrement dit, son dénominateur est égal à 1.
L'absence de pôle est une particularité des filtres RIF,
et comme ils ne risquent pas d'avoir de pôles à l'extérieur du cercle unité, ils sont toujours stables !
Les filtres RIF sont parfois appelés filtres à moyenne mobile ou MA (_moving average_).

### Schéma bloc

Les filtres numériques sont souvent représentés sous forme d'un schéma bloc,
qui illustre graphiquement les relations entre l'entrée et la sortie du filtre.
Pour cela, il convient d'exprimer la sortie en fonction de l'entrée.
Comme on sait que la fonction de transfert $H(z)$ est le rapport entre la sortie sur l'entrée :

$$
H(z) = \frac{Y(z)}{X(z)},
$$

alors :

$$
\sum_{n=0}^{N} b_n z^{-n} = \frac{Y(z)}{X(z)}
\qquad\Leftrightarrow\qquad
Y(z) = \sum_{n=0}^{N} b_n z^{-n} X(z)
$$

dont la transformée en z inverse est, grâce à la propriété de linéarité :

```{margin}
Remarquez qu'on retombe avec cette formule sur le produit de convolution !
```

$$
y[n] &= \sum_{k=0}^{N} b_k x[n-k] \\
     &= b_0 x[n] + b_1 x[n-1] + b_2 x[n-2] + \dots + b_N x[n-N].
$$

La sortie $y[n]$ est donc la somme de $N+1$ échantillons d'entrée pondérés par les coefficients $b_k$
et retardés de $k$ échantillons.
On peut donc adopter la représentation suivante, où le bloc $z^{-1}$ indique un retard :

```{figure} figs/filtre-rif.svg
:height: 350px
```

Contrairement aux schémas-blocs des filtres RII que l'on verra plus loin,
il n'y a pas de boucle de retour pour les filtres RIF : on dit qu'ils sont non récursifs.
Par conséquent, la sortie ne peut pas être réinjectée indéfiniment dans le filtre,
empêchant toute instabilité.


### Linéarité de la phase

La linéarité de la phase est souvent un critère souhaité.
Les filtres RIF sont à phase linéaire si leur réponse impulsionnelle est symétrique
(attention, symétrique ne veut pas dire paire ici !) ou antisymétrique, c'est-à-dire :

$$
\forall n\in\{0,\dots,N\}\quad
&h[n] = h[N-1-n]  &\quad\text{pour un filtre symétrique,} \\
&h[n] = -h[N-1-n] &\quad\text{pour un filtre antisymétrique.}
$$

La {numref}`F:num:ri` illustre ces propriétés.

```{figure} figs/rif-types.svg
---
name: F:num:ri
---
Exemples de réponses impulsionnelles symétriques (types I et II) et antisymétriques (types III et IV).
```

Notons que les filtres de type II, III et IV ne sont pas adaptés pour synthétiser des filtres passe-haut.


### Une méthode toute simple pour synthétiser un RIF

<!-- [Méthode des fenêtres](https://vincmazet.github.io/signal2/_static/files/rif.pdf) -->

Une idée toute simple pour obtenir un filtre RIF est de tronquer la réponse impulsionnelle d'un filtre idéal.

Prenons l'exemple d'un filtre passe-bas, tel qu'on l'a vu dans la section {ref}`S:filtre-intro:pbas`.
Sa réponse fréquentielle est (pour simplifier l'exposé, on prend $K=1$ et $a=0$) :

$$
H_\text{idéal}(f) = \begin{cases}
  1 &\text{si}\, |f| \leq f_c, \\
  0 &\text{si}\, f_c < |f| < f_e/2.
\end{cases}
$$

Remarquez qu'on a ajouté la condition $|f| < f_e/2$ puisque le filtre étant numérique
et d'après le théorème de l'échantillonnage, il n'y a pas de fréquence au delà de la fréquence de Nyquist.

La transformée de Fourier à temps discret inverse de $H_\text{idéal}(f)$ est un sinus cardinal :

$$
h_\text{idéal}[n] = 2 f_c \mathrm{sinc}(2f_c n).
$$

En ne conservant que $N+1$ échantillons, on obtient forcément un filtre de réponse impulsionnelle finie.
On choisit pour cela de tronquer $h_\text{idéal}$ autour de l'échantillon $n=0$,
et de décaler les échantillons conservés de sorte à rendre le filtre causal.
Ce nouveau filtre a pour réponse impulsionnelle :

$$
h =
\begin{cases}
2 f_c \mathrm{sinc}(2f_c (n-N/2)) &\text{si $n \in \{0,\dots,N\}$} \\
0                               &\text{sinon}.
\end{cases}
$$

```{figure} figs/fenetre-troncature.svg
:name: F:num:fenetre-troncature
:width: 700px

Troncature et décalage de la réponse impulsionnelle idéale.
Vous noterez que ce filtre est de type I.
```

Enfin, la transformée de Fourier à temps discret de $h$ nous permet d'obtenir la réponse fréquentielle du filtre.
Son expression est assez complexe et nous ne la détaillons pas ici.
La figure {numref}`F:num:rif-rf` représente le gain de ce filtre RIF construit à partir de la troncature d'un filtre idéal.

```{figure} figs/rif-rf.svg
:name: F:num:rif-rf
:width: 700px

Réponse fréquentielle du filtre tronqué et décalé.
```

On observe plusieurs choses sur cette figure :
* il y a des oscillations en bande passante et en bande atténuée,
* la bande de transition n'est pas nulle.

Les oscillations sont d'ailleurs de plus en plus grandes lorsqu'on s'approche de la bande de transition :
c'est le phénomène de Gibbs.
Il s'avère que ces oscillations ne s'atténuent pas lorsque $N$ augmente,
alors que pourtant le filtre RIF se rapproche de plus en plus du filtre idéal !

L'interprétation que l'on peut faire de ces observations est la suivante.
La troncature revient à multiplier la réponse impulsionnelle $h_\text{idéal}$ par une porte $w[n]$
(on parle de « fenêtrage rectangulaire ») :

$$
h[n] = h_\text{idéal}[n] \times w[n].
$$

Dans le domaine de Fourier, cela revient à convoluer la réponse fréquentielle $H_\text{idéal}$
par $W(f)$ qui est la transformée de Fourier à temps discret de $w[n]$ :

$$
H(f) = H_\text{idéal}(f) * W(f),
$$

où :

$$
W(f) = e^{-j \pi f N} \frac{\sin(\pi f(N+1))}{\sin(\pi f)}.
$$

$W(f)$ est la multiplication d'une exponentielle avec un « noyau de Dirichlet » (cf. {numref}`F:num:dirichlet`).

```{figure} figs/dirichlet.svg
:name: F:num:dirichlet
:width: 700px

Noyau de Dirichlet.
```

On observe que lorsque $N$ augmente, le lobe principal du noyau de Dirichlet se rétrécit,
et donc la bande de transition du filtre également.
Obtenir des bandes de transition très faibles oblige donc à augmenter considérablement l'ordre du filtre $N$,
résultant alors dans un filtre à réponse impulsionnelle pas tant finie que ça...

(S:methode-des-fenetres)=
### Méthode des fenêtres

```{margin}
Et oui, en traitement du signal, la porte est un genre de fenêtre !
```

La méthode des fenêtres reprend l'idée précédente (troncature de la réponse impulsionnelle d'un filtre idéal)
mais la fenêtre n'est pas forcément une porte.

Le tableau ci-dessous donne les propriétés des réponses fréquentielles obtenues pour plusieurs fenêtres
(la fréquence réduite est la fréquence divisée par $f_e$).

| Fenêtre | Largeur de transition (fréquence réduite) | Ondulation en bande passante (dB) | Atténuation mminimale (dB) |
| :--- | --- | --- | ---: |
| Rectangulaire        | $0,9/N$               | $0,742$             | $21$ |
| Hamming              | $3,3/N$               | $0,019$             | $53$ |
| Blackman             | $5,5/N$               | $0,002$             | $74$ |
| Kaiser $\beta=4,538$ | $2,93/N$              | $0,0274$            | $50$ |
| Kaiser $\beta=6,764$ | $4,32/N$              | $0,00275$           | $70$ |
| Kaiser $\beta=8,960$ | $5,71/N$              | $0,000275$          | $90$ |

<!-- Figure : Définition des différents éléments de la réponse fréquentielle ? Mais redondante avec ce qu'on a déjà vu -->

<!-- https://github.com/scipy/scipy/blob/v1.7.1/scipy/signal/fir_filter_design.py#L48-L83 kaiser_beta(a) -->

### Autres méthodes de synthèse

La méthode des fenêtres peut donner des ordres de filtres surévalués.
D'autres méthodes de synthèse existe, comme par exemple la méthode de Parks-McClelland
qui ajuste les coefficients de manière itérative pour que l'écart au filtre idéal soit inférieure à une valeur limite.
La méthode est accessible avec la fonction `scipy.signal.remez` (car elle est une variation de l'algorithme de Remez).

<!--
http://eeweb.poly.edu/iselesni/EL713/remez/remez.pdf
https://en.wikipedia.org/wiki/Parks%E2%80%93McClellan_filter_design_algorithm
http://www.ieee.org.ar/downloads/antoniou-nov03.pdf -->

<!-- Equiripple FIR filters ? least-squares error minimization ? -->

## Filtres RII

### Réponse impulsionnelle

Un filtre RII a une réponse impulsionnelle qui ne s'annule jamais au délà d'un certain échantillon :

$$
h[n] =
\begin{cases}
c_n &\text{si}\, n \geq 0, \\
0   &\text{sinon}.
\end{cases}
$$

### Fonction de transfert

La fonction de transfert d'un filtre RII est de la forme:

$$
H(z) = \frac{ \sum_{m=0}^M b_m z^{-m} }{ \sum_{n=0}^N a_n z^{-n} }.
$$

Les filtres RII sont parfois appelés filtres ARMA car la fonction de transfert est le quotient entre :
* un numérateur correspondant à un filtre MA (on l'a vu plus haut),
* et un dénominateur correspondant à un filtre appelé AR (_autoregressive_).

### Schéma bloc

Comme pour les filtres RIF, il est possible de représenter les filtres RII sous forme d'un schéma bloc.
À partir de la fonction de transfert, on peut écrire la sortie en fonction de l'entrée :

$$
H(z) = \frac{Y(z)}{X(z)} = \frac{ \sum_{m=0}^M b_m z^{-m} }{ \sum_{n=0}^N a_n z^{-n} }
\qquad\Leftrightarrow\quad
\sum_{k=0}^N a_k z^{-k} Y(z) = \sum_{l=0}^M b_l z^{-l} X(z).
$$

d'où, en utilisant la transformée en z inverse :

$$
\sum_{k=0}^N a_k y[n-k] = \sum_{l=0}^M b_l x[n-l].
$$

On peut alors isoler $y[n]$ et l'écrire en fonction des autres termes :

$$
y[n] = \frac{1}{a_0} \left( \sum_{l=0}^M b_l x[n-l] - \sum_{k=1}^N a_k y[n-k] \right).
$$

La sortie $y[n]$ est donc la somme de $M+1$ échantillons d'entrée pondérés et retardés
et de $N$ échantillons de la sortie pondérés et retardés,
le tout multiplié par $\frac{1}{a_0}$.
On peut donc représenter graphiquement cette équation (rappel : le bloc $z^{-1}$ indique un retard) :

```{figure} figs/filtre-rii.svg
:height: 400px
```

Remarquez qu'à l'inverse des filtres RIF, les filtres RII possèdent une boucle de retour (partie récursive).
À cause de cette boucle, les filtres RII peuvent être instables :
le signal étant indéfiniment réinjecté dans le filtre, la sortie peut être de plus en plus grande.


### Transformation bilinéaire

La transformation bilinéaire est la méthode la plus simple pour concevoir un filtre RII.
Elle s'applique sur un filtre analogique pour le transformer en filtre numérique.
De ce fait, l'application de la transformation bilinéaire suit les étapes ci-dessous,
en partant d'un gabarit numérique.

1. Les fréquences $f_n$ du gabarit numérique sont remplacées par de nouvelles valeurs $f_a$ qui définissent un gabarit analogique.
   La transformation de fréquence est :
   
   $$
   f_a = \frac{f_e}{\pi} \tan\left(\pi\frac{f_n}{f_e}\right).
   $$
   
   où $f_e$ est la fréquence d'échantillonnage.

1. Un [filtre analogique](C:filtre-analogique) est conçu à partir de ce gabarit analogique :
   on dispose alors de la fonction de transfert $H_a(s)$ de ce filtre analogique.

1. Enfin, la fonction de transfert numérique $H_n(z)$ s'obtient à partir de la la fonction de transfert analogique $H_a(s)$
   grâce à la relation suivante :

   $$
   H_n(z) = H_a\left(2f_e \left(\frac{1-z^{-1}}{1+z^{-1}}\right)\right).
   $$

Cette méthode se base sur la transformation bilinéaire qui effectue le passage du domaine analogique au domaine numérique
en transformant l'axe des imaginaires du plan en $s$ en le cercle unité du plan en $z$ :

$$
s = \frac{2}{T_e} \left(\frac{1-z^{-1}}{1+z^{-1}}\right)
\qquad\Leftrightarrow\qquad
z = \frac{1+T_e/2s}{1-T_e/2s}.
$$

La {numref}`F:num:tbilin` illustre cette transformation, ainsi que ces deux vidéos :
<a href="https://www.youtube.com/watch?v=4PV6ikgBShw"><i class="fab fa-youtube"></i></a> et
<a href="https://www.youtube.com/watch?v=acQecd6dmxw"><i class="fab fa-youtube"></i></a>.

```{figure} figs/tbilineaire.svg
:name: F:num:tbilin
:width: 500px

Transformée bilinéaire.
```

Ainsi, la stabilité du filtre analogique est conservée pour le filtre numérique
puisque les pôles à partie réelle négative du plan en $s$ se retrouvent dans le cercle unité du plan en $z$.

    


<!-- Lorsque l'ordre des filtres augmente (c'est-à-dire lorsque le nombre de coefficients augmentent) le filtre peut devenir instable à cause de la quantification des coefficients point. Une solution consiste à remplacer le filtre par une série de filtres d'ordre faible (un ou 2) 

Filtres, bi-quad, biquadratique 

SOS s order section 
-->

<!-- autres méthodes de conception de filtres RII : design direct (approximer par une réponse fréquentielle linéaire par morceaux ; Yulewalk), Butterworth généralisé (plus de zéros ; maxflat), modèle paramétrique ? (cf Matlpab : lpc, prony, invfreqs) -->